In [1]:
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from sympy import sympify
import sys; sys.path.insert(0, "../"); from utils import *
from models import SympyTorch, PartialDerivativeCalculator, CoeffLearner
from parametric_discovery_pinn import ParametricPINN, BurgerPINN, FinalParametricPINN
from madgrad import MADGRAD

from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from robust_pde_diff import print_pde, RobustPCA, Robust_LRSTR
from parametric_pde_diff import TrainSGTRidge, create_groups

from scipy.integrate import odeint
from numpy.fft import fft, ifft, fftfreq
from time import time

from pysr import pysr, best

fontsize = 20

Populating the interactive namespace from numpy and matplotlib
Running Python 3.9.6
You can use npar for np.array


In [2]:
data = pickle_load("../parametric_pde_data/parametric_burgers.pkl")

x = data['x']; spatial_dims = x.shape[0]
t = data['t']; time_dims = t.shape[0]

Exact = data['u']
X, T = np.meshgrid(x, t)

# Adding noise
noise_intensity = 0.01
noisy_xt = True

Exact = perturb(Exact, intensity=noise_intensity, noise_type="normal")
print("Perturbed Exact with intensity =", float(noise_intensity))

X_star = np.hstack((to_column_vector(X), to_column_vector(T)))
u_star = to_column_vector(Exact.T)

# Add noise to (x, t) before setting the lb, and ub.
if noisy_xt: 
    print("Noisy (x, t)")
    X_star = perturb(X_star, intensity=noise_intensity, noise_type="normal")
else: print("Clean (x, t)")

# domain bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

# Sampling training data points
N = 20000
training_idxs = sampling_from_rows(X_star, N, True)
X_train = X_star[training_idxs, :]
u_train = u_star[training_idxs, :]

# to_tensor
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
lb = to_tensor(lb, False)
ub = to_tensor(ub, False)

u_xx_true = 0.1*np.ones(time_dims)
uu_x_true = -1*(1+0.25*np.sin(t))

feature_names = ['u', 'u_x', 'u_xx']

Loaded from ../parametric_pde_data/parametric_burgers.pkl
Perturbed Exact with intensity = 0.01
Noisy (x, t)


In [3]:
pinn = ParametricPINN(scale=False, lb=lb, ub=ub)
print("Loaded the pretrained weights")
pinn.load_state_dict(torch.load("./saved_path_inverse_parametric_burgers/parametric_pinn.pth"))
model = nn.Sequential(pinn.preprocessor_net, pinn.pde_solver_net)

pde_terms = ["u*u_x", "u_xx"]
func_terms = ["-0.1872898*sin(t)-1.0238724", "0.09875935"]
final_burger_pinn = FinalParametricPINN(model=model, pde_terms=pde_terms, func_terms=func_terms, uncert=True, scale=pinn.scale, lb=pinn.lb, ub=pinn.ub)
del pinn

Loaded the pretrained weights


In [4]:
x, t = dimension_slicing(X_train)

In [5]:
def pcgrad_closure(return_list=False):
    global N, x, t, u_train
    losses = final_burger_pinn.loss(x, t, u_train)
    updated_grads = []
    
    for i in range(2):
        optimizer.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in final_burger_pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(final_burger_pinn.parameters()):
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    if not return_list: return losses[0]+losses[1]
    else: return losses

In [6]:
def finetuning_closure():
    global N, x, t, u_train
    if torch.is_grad_enabled(): f_opt.zero_grad()
    # the solver network only consider the first N samples.
    mse_loss, pde_loss = final_burger_pinn.loss(x, t, u_train)
    loss = mse_loss + pde_loss
    if loss.requires_grad: loss.backward(retain_graph=False)
    return loss

In [7]:
optimizer = MADGRAD(final_burger_pinn.parameters(), lr=1e-6, momentum=0.9)
for i in range(150):
    final_burger_pinn.train()
    optimizer.step(pcgrad_closure)
    if i%10==0: print(pcgrad_closure(return_list=True))
        
f_opt = torch.optim.LBFGS(final_burger_pinn.parameters(), lr=1e-1, max_iter=500, max_eval=int(1.25*500), history_size=300, line_search_fn='strong_wolfe')
final_burger_pinn.is_uncert = True
for i in range(100):
    f_opt.step(finetuning_closure)
    if i%10==0: print(finetuning_closure())

[tensor(0.6932, grad_fn=<AddBackward0>), tensor(0.6931, grad_fn=<AddBackward0>)]
[tensor(0.6926, grad_fn=<AddBackward0>), tensor(0.6925, grad_fn=<AddBackward0>)]
[tensor(0.6916, grad_fn=<AddBackward0>), tensor(0.6915, grad_fn=<AddBackward0>)]
[tensor(0.6904, grad_fn=<AddBackward0>), tensor(0.6904, grad_fn=<AddBackward0>)]
[tensor(0.6892, grad_fn=<AddBackward0>), tensor(0.6892, grad_fn=<AddBackward0>)]
[tensor(0.6880, grad_fn=<AddBackward0>), tensor(0.6880, grad_fn=<AddBackward0>)]
[tensor(0.6868, grad_fn=<AddBackward0>), tensor(0.6868, grad_fn=<AddBackward0>)]
[tensor(0.6856, grad_fn=<AddBackward0>), tensor(0.6856, grad_fn=<AddBackward0>)]
[tensor(0.6844, grad_fn=<AddBackward0>), tensor(0.6844, grad_fn=<AddBackward0>)]
[tensor(0.6832, grad_fn=<AddBackward0>), tensor(0.6832, grad_fn=<AddBackward0>)]
[tensor(0.6820, grad_fn=<AddBackward0>), tensor(0.6820, grad_fn=<AddBackward0>)]
[tensor(0.6808, grad_fn=<AddBackward0>), tensor(0.6807, grad_fn=<AddBackward0>)]
[tensor(0.6796, grad_fn=<Add

In [8]:
[x.item() for x in final_burger_pinn.pdc.parameters()]

[-1.0028020143508911, -0.24078042805194855, 0.10122136771678925]

In [9]:
errs = np.array([100*(1.0028020143508911-1), 100*(0.25-0.24078042805194855)/0.25, 100*(0.10122136771678925-0.1)/0.1])
errs.mean(), errs.std()

(1.729799310366311, 1.4368618683618857)

In [10]:
# Clean Exact and clean (x, t) | final_parametric_pinn.pth
# params: [-1.0013394355773926, -0.2480146884918213, 0.0997583195567131]
# errs: (0.38991620143254785, 0.28918291006992664)

# Noisy Exact and clean (x, t)
# params: [-1.0029208660125732, -0.24306637048721313, 0.09983817487955093]
# errs: (1.075787842273714, 1.2016070382040953)

# Noisy Exact and Noisy (x, t)
# params: [-1.0028020143508911, -0.24078042805194855, 0.10122136771678925]
# errs: (1.729799310366311, 1.4368618683618857)

In [11]:
# save(final_burger_pinn, "./saved_path_inverse_parametric_burgers/noisy2_final_parametric_pinn.pth")